In [ ]:
from mysql.connector import pooling
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances
import spacy
import pandas as pd
import os
from collections import namedtuple
from dotenv import load_dotenv
load_dotenv()

In [ ]:
ImageProperties = namedtuple('ImageProperties', ['name', 'hex_color', 'tags', 'make', 'orientation', 'width', 'height'])

In [ ]:
preferences = {
    'Make': '',
    'ImageWidth': '',
    'ImageHeight': '',
    'Orientation': 1,
    'dominant_color': '#73AD3D',
    'tags': ['vase', 'toilet']
}

# Set SQL variables
sql_host = os.getenv("SQL_HOST")
sql_user = os.getenv("SQL_USER")
sql_password = os.getenv("SQL_PASSWORD")
sql_database = os.getenv("SQL_DATABASE")

# set the database config
config = {
    'user': sql_user,
    'password': sql_password,
    'host': sql_host,
    'port': '3306',
    'database': sql_database,
}

In [ ]:
# Create a connection pool
connection_pool = pooling.MySQLConnectionPool(pool_name="mypool",
                                              pool_size=2,
                                              **config)

In [ ]:
def get_metadata_from_mariadb_db():
    """
    Get the metadata from the MariaDB database

    :return: A pandas DataFrame with the metadata
    """
    # Open a connection to the database
    conn = connection_pool.get_connection()
    # Create a cursor
    c = conn.cursor()

    # Retrieve the metadata
    c.execute("""
        SELECT filename, GROUP_CONCAT(CONCAT(mkey, '\t', mvalue) SEPARATOR '\n') AS metadata
        FROM metadata
        GROUP BY filename;
    """)
    metadata = c.fetchall()

    # Close the connection
    conn.close()

    # Create an empty DataFrame with the desired columns
    columns = ['filename', 'Make', 'Software', 'ImageWidth', 'ImageHeight', 'Orientation', 'DateTimeOriginal',
               'dominant_color', 'tags']
    df = pd.DataFrame(columns=columns)

    # Fill the DataFrame with the metadata
    for image in tqdm(metadata, desc="Get metadata from database"):
        try:
            props = {'filename': image[0]}
            metadata_str = image[1].split('\n')
            for prop in metadata_str:
                if prop:
                    k, value = prop.split('\t')
                    if k in columns[1:]:
                        if k == 'dominant_color':
                            color_list = eval(value)
                            color_list = [c[0] for c in color_list]
                            props[k] = color_list
                        elif k == 'tags':
                            props[k] = eval(value)
                        else:
                            props[k] = value
            df = df.append(props, ignore_index=True)
        except Exception as e:
            print(e, image)

    return df

import ast

ImageProperties = namedtuple('ImageProperties', ['name', 'hex_color', 'tags', 'make', 'orientation', 'width', 'height'])

def get_metadata_from_mariadb_as_imageproperties():
    # Open a connection to the database
    conn = connection_pool.get_connection()
    # Create a cursor
    c = conn.cursor()

    # Retrieve the metadata
    c.execute("""
        SELECT DISTINCT filename, GROUP_CONCAT(CONCAT(mkey, '\t', mvalue) SEPARATOR '\n') AS metadata
        FROM metadata
        WHERE mkey IN ('Make', 'Orientation', 'ImageWidth', 'ImageHeight', 'tags', 'dominant_color')
        GROUP BY filename;
    """)
    metadata = c.fetchall()

    # Close the connection
    conn.close()

    # use the namedtuple ImageProperties to store the metadata
    images = []

    # Loop through the rows of metadata
    for row in metadata:
        filename, metadata_str = row
        metadata_items = metadata_str.split('\n')
        metadata_dict = {key: value for key, value in (item.split('\t') for item in metadata_items)}

        # Clean dominant colors: convert the string to a list of tuples and extract only the color hex codes
        dominant_colors = ast.literal_eval(metadata_dict.get('dominant_color', '[]'))
        hex_colors = [color[0] for color in dominant_colors]

        # Clean tags: convert the string to a list of strings
        tags = ast.literal_eval(metadata_dict.get('tags', '[]'))

        # Create an ImageProperties object for each row
        image = ImageProperties(
            name=filename,
            hex_color=hex_colors,
            tags=tags,
            make=metadata_dict.get('Make', None),
            orientation=metadata_dict.get('Orientation', None),
            width=metadata_dict.get('ImageWidth', None),
            height=metadata_dict.get('ImageHeight', None)
        )

        # Add the ImageProperties object to the list
        images.append(image)

    return images

In [ ]:
def hex_to_rgb(color):
    try:
        # remove the # from the color
        color = color[1:]
        # convert the color to rgb values
        rgb = tuple(int(color[i:i + 2], 16) for i in (0, 2, 4))
        return rgb
    except:
        return 0, 0, 0


def get_clean_preferences(df_preferences):
    # remove the rows with nan in dominant_color
    df_preferences = df_preferences.dropna(subset=['dominant_color'])
    # split dominant color into 4 columns and remove the dominant_color column
    # convert the tags column to a list of strings
    # Replace all NaN values with empty strings with the fillna() method
    df_preferences = df_preferences.fillna(0)
    # convert colors to rgb values
    df_preferences['dominant_color'] = df_preferences['dominant_color'].apply(lambda x: hex_to_rgb(x))
    # replace all 0 values with empty strings
    df_preferences['dominant_color'] = df_preferences['dominant_color'].replace(0, '')

    return df_preferences


def get_clean_dataset():
    metadata = get_metadata_from_mariadb_db()
    df_metadata = pd.DataFrame(metadata)
    # remove the rows with nan in dominant_color
    df_metadata = df_metadata.dropna(subset=['dominant_color'])
    # split dominant color into 4 columns and remove the dominant_color column
    if 'dominant_color' in df_metadata.columns:
        df_metadata['color1'] = df_metadata['dominant_color'].apply(lambda x: x[0] if len(x) >= 1 else 0)
        df_metadata['color2'] = df_metadata['dominant_color'].apply(lambda x: x[1] if len(x) >= 2 else 0)
        df_metadata['color3'] = df_metadata['dominant_color'].apply(lambda x: x[2] if len(x) == 3 else 0)
        df_metadata['color4'] = df_metadata['dominant_color'].apply(lambda x: x[3] if len(x) == 4 else 0)
        # convert colors to rgb values
        df_metadata['color1'] = df_metadata['color1'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color2'] = df_metadata['color2'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color3'] = df_metadata['color3'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata['color4'] = df_metadata['color4'].apply(lambda x: hex_to_rgb(x) if x else (0, 0, 0))
        df_metadata = df_metadata.drop('dominant_color', axis=1)
    else:
        df_metadata['color1'] = 0
        df_metadata['color2'] = 0
        df_metadata['color3'] = 0
        df_metadata['color4'] = 0

    # convert the tags column to a list of strings
    df_metadata = df_metadata.fillna(0)
    # remove all columns except filename, tags, color1, color2, color3, color4, Make, Width, Height
    df_metadata = df_metadata[
        ['filename', 'Make', 'ImageWidth', 'ImageHeight', 'Orientation', 'DateTimeOriginal', 'tags', 'color1', 'color2',
         'color3', 'color4']]
    # replace all 0 values with empty strings
    df_metadata['Make'] = df_metadata['Make'].replace(0, '')

    return df_metadata

In [ ]:
df_pref = pd.DataFrame([preferences])
df_preferences = get_clean_preferences(df_pref)
df_preferences.head()

In [ ]:
df_metadata = get_clean_dataset()

In [ ]:
df_metadata.head()

# Color Similarity

In [ ]:
def recommend_colors(df_metadata, df_preferences, n=0):
    # Load the dataset into a Pandas DataFrame
    data = df_metadata.copy()

    # Extract the individual r, g, and b values from tupbles in the color columns
    data[['r1', 'g1', 'b1']] = pd.DataFrame(data['color1'].tolist(), index=data.index)
    data[['r2', 'g2', 'b2']] = pd.DataFrame(data['color2'].tolist(), index=data.index)
    data[['r3', 'g3', 'b3']] = pd.DataFrame(data['color3'].tolist(), index=data.index)
    data[['r4', 'g4', 'b4']] = pd.DataFrame(data['color4'].tolist(), index=data.index)

    # Normalize the r, g, and b columns to be between 0 and 1
    data[['r1', 'g1', 'b1', 'r2', 'g2', 'b2', 'r3', 'g3', 'b3', 'r4', 'g4', 'b4']] = data[['r1', 'g1', 'b1', 'r2', 'g2',
                                                                                           'b2', 'r3', 'g3', 'b3', 'r4',
                                                                                           'g4', 'b4']] / 255

    # Normalize the input RGB color to be between 0 and 1
    r, g, b = df_preferences['dominant_color'][0]
    r_norm, g_norm, b_norm = r / 255, g / 255, b / 255

    # Compute the Euclidean distance between the input color and all the colors in the dataset
    data['similarity_dominant_color'] = euclidean_distances(
        [[r_norm, g_norm, b_norm, r_norm, g_norm, b_norm, r_norm, g_norm, b_norm, r_norm, g_norm, b_norm]],
        data[['r1', 'g1', 'b1', 'r2', 'g2', 'b2', 'r3', 'g3', 'b3', 'r4', 'g4', 'b4']])[0]

    # Sort the dataset by Euclidean distance in ascending order and return the top 10 closest matches
    if n == 0:
        closest_matches = data.sort_values('similarity_dominant_color', ascending=True)[
            ['filename', 'color1', 'color2', 'color3', 'color4', 'similarity_dominant_color']]
    else:
        closest_matches = data.sort_values('similarity_dominant_color', ascending=True).head(n)[
            ['filename', 'color1', 'color2', 'color3', 'color4', 'similarity_dominant_color']]

    return closest_matches

In [ ]:
recommend_colors(df_metadata, df_preferences)  # OK

# Tag Similarity

In [ ]:
def recommend_tags(df_metadata, df_preferences, n=0, nlp=None):
    # Load the spaCy model if it hasn't been loaded
    if not nlp:
        nlp = spacy.load("en_core_web_md")

    # Define the preferences list and the dataframe
    preferences = df_preferences['tags'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.copy()
    df = df.dropna(subset=["tags"]).reset_index(drop=True)
    # replace int with empty list
    df['tags'] = df['tags'].apply(lambda x: x if x else [])

    # Precompute the similarity between each tag word and each preference word
    similarity_dict = {}
    for tag_word in set([word for tags in df['tags'] for word in tags]):
        for pref_word in set(preferences):
            similarity_dict[(tag_word, pref_word)] = nlp(tag_word).similarity(nlp(pref_word))

    # Compute the average similarity for each row in the dataframe
    similarities = []
    for tags in df['tags']:
        sum_similarity = 0
        for tag_word in tags:
            for pref_word in preferences:
                sum_similarity += similarity_dict[(tag_word, pref_word)]
        avg_similarity = sum_similarity / (len(tags) * len(preferences)) if len(tags) > 0 else 0
        similarities.append(avg_similarity)

    # Add the similarity scores to a new column in the dataframe
    df['similarity_tags'] = similarities
    if n == 0:
        closest_matches = df.sort_values('similarity_tags', ascending=False)[
            ['filename', 'similarity_tags']]
    else:
        closest_matches = df.sort_values('similarity_tags', ascending=False).head(n)[
            ['filename', 'similarity_tags']]

    return closest_matches


In [ ]:
recommend_tags(df_metadata, df_preferences)  # OK

# Make Similarity

In [ ]:
def recommend_make(df_metadata, df_preferences, n=0):
    # Load the spaCy model
    nlp = spacy.load("en_core_web_md")

    # Define the preferences list and the dataframe
    make = df_preferences['Make'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.copy()
    df = df.dropna(subset=["Make"]).reset_index(drop=True)

    # Convert make and Make to document objects
    make_doc = nlp(make)
    df['Make'] = df['Make'].apply(nlp)

    # Compute the cosine similarity between the make preferences and all the makes in the dataset
    similarities = [make_doc.similarity(doc) for doc in df['Make']]

    # Add the similarity scores to a new column in the dataframe
    df['similarity_make'] = similarities
    if n == 0:
        closest_matches = df.sort_values('similarity_make', ascending=False)[
            ['filename', 'similarity_make']]
    else:
        closest_matches = df.sort_values('similarity_make', ascending=False).head(n)[
            ['filename', 'similarity_make']]

    return closest_matches


In [ ]:
recommend_make(df_metadata, df_preferences)  # OK

# Orientation Similarity

In [ ]:
def recommend_orientation(df_metadata, df_preferences, n=0):
    # Define the preferences list and the dataframe
    orientation = df_preferences['Orientation'][0]
    # Load dataset with words and drop duplicate rows
    df = df_metadata.dropna(subset=["Orientation"]).reset_index(drop=True)
    # if Orientation contain '' or '0' or '1' then replace with 0 or 1
    df['Orientation'] = df['Orientation'].apply(lambda x: 0 if x == '' or x == '0' else 1)

    # Convert the Orientation column to integer type
    df['Orientation'] = df['Orientation'].astype(int)

    # Orientation is 0 or 1, so we can just subtract the preference from the orientation
    df['similarity_orientation'] = df['Orientation'].apply(lambda x: abs(x - orientation))

    # sort by similarity
    if n > 0:
        closest_matches = df.sort_values('similarity_orientation', ascending=False).head(n)[
            ['filename', 'similarity_orientation']]
    else:
        closest_matches = df.sort_values('similarity_orientation', ascending=False)[
            ['filename', 'similarity_orientation']]

    return closest_matches


In [ ]:
recommend_orientation(df_metadata, df_preferences)  # OK

# Size Similarity

In [ ]:
def recommend_size(df_metadata, df_preferences, n=0):
    # Define the preferences list and the dataframe
    width = int(df_preferences['ImageWidth'][0])
    height = int(df_preferences['ImageHeight'][0])
    # Load dataset with words and drop duplicate rows
    df = df_metadata.dropna(subset=["ImageWidth", "ImageHeight"]).reset_index(drop=True)

    # Convert the ImageWidth and ImageHeight column to integer type
    df[['ImageWidth', 'ImageHeight']] = df[['ImageWidth', 'ImageHeight']].astype(int)

    # Compute the product of width and height outside the loop
    product = width * height

    # Use apply method to compute similarity score for each row
    df['similarity_size'] = df.apply(lambda x: 1 - abs(product - (x['ImageWidth'] * x['ImageHeight'])) / product, axis=1)

    if n == 0:
        closest_matches = df.sort_values('similarity_size', ascending=False)[
            ['filename', 'similarity_size']]
    else:
        closest_matches = df.sort_values('similarity_size', ascending=False).head(n)[
            ['filename', 'similarity_size']]

    return closest_matches


In [ ]:
recommend_size(df_metadata, df_preferences)  # OK

In [ ]:
def recommend(df_metadata, df_preferences, n=0):
    # Assign weights to properties based on user preferences
    weights = {
        'Make': float(5.0),
        'ImageWidth': float(1.0),
        'ImageHeight': float(1.0),
        'Orientation': float(2.0),
        'dominant_color': float(3.0),
        'tags': float(5.0)
    }

    # Create a dictionary with the preferences and the corresponding recommendation methods
    preference_methods = {
        'Make': recommend_make,
        'ImageWidth': recommend_size,
        'ImageHeight': recommend_size,
        'Orientation': recommend_orientation,
        'dominant_color': recommend_colors,
        'tags': recommend_tags
    }

    # Remove preferences with no values
    preferences = {k: v for k, v in df_preferences.squeeze().to_dict().items() if v != ''}

    # Calculate the sum of the weights
    weights_sum = 0
    for key in weights:
        weights_sum += weights[key]
    for key in weights:
        weights[key] = weights[key] / weights_sum

    # Calculate similarity score for each property
    df_metadata['similarity_score'] = 0.0
    for preference, value in preferences.items():
        method = preference_methods[preference]
        if preference == 'ImageWidth' or preference == 'ImageHeight':
            similarity = method(df_metadata, df_preferences, n)['similarity_size'].astype(float)
        else:
            similarity = method(df_metadata, df_preferences, n)[f'similarity_{preference.lower()}'].astype(float)
        df_metadata['similarity_score'] += similarity * (weights[preference] / weights_sum)

    # Replace NaN values in the 'similarity_score' column with 0
    df_metadata['similarity_score'].fillna(0, inplace=True)

    # Sort by similarity score
    if n == 0:
        closest_matches = df_metadata.sort_values('similarity_score', ascending=False)[
            ['filename', 'similarity_score']]
    else:
        closest_matches = df_metadata.sort_values('similarity_score', ascending=False).head(n)[
            ['filename', 'similarity_score']]

    return closest_matches


In [ ]:
recommend(df_metadata, df_preferences)  # OK



# Test


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = get_metadata_from_mariadb_as_imageproperties()

preferences = {
    'Make': 'Canon',
    'ImageWidth': '',
    'ImageHeight': '',
    'Orientation': 1,
    'dominant_color': '#00000',
    'tags': ['cat']
}


def hex_to_rgb(hex_color):
    return tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))


def preprocess_data(data):
    for image in data:
        image_tags = " ".join(image.tags)
        avg_rgb_color = np.mean([hex_to_rgb(color) for color in image.hex_color], axis=0)
        try:
            make_len = len(image.make)
        except TypeError:
            make_len = 0

        try:
            width = int(image.width)
        except TypeError:
            width = 0

        try:
            height = int(image.height)
        except TypeError:
            height = 0

        try:
            orientation = int(image.orientation)
        except:
            orientation = 0

        avg_rgb_color_list = avg_rgb_color.tolist() if hasattr(avg_rgb_color, 'tolist') else [avg_rgb_color]

        try:
            yield np.array([
                *avg_rgb_color_list,  # Ensure avg_rgb_color is an iterable before unpacking
                len(image_tags),
                make_len,
                orientation,
                width,
                height
            ])
        except:
            pass


def user_preferences_vector(preferences):
    dominant_color = hex_to_rgb(preferences['dominant_color'])
    tags = " ".join(preferences['tags'])
    make = preferences['Make']

    try:
        make_len = len(make)
    except TypeError:
        make_len = 0

    orientation = preferences['Orientation']

    try:
        width = int(preferences['ImageWidth'])
    except (TypeError, ValueError):
        width = 0

    try:
        height = int(preferences['ImageHeight'])
    except (TypeError, ValueError):
        height = 0

    return np.array([
        *dominant_color,
        len(tags),
        make_len,
        orientation,
        width,
        height
    ])


def recommend_images(preferences, data, top_n=10):
    preprocessed_data = np.array(list(preprocess_data(data)))
    user_vector = user_preferences_vector(preferences)
    similarity_matrix = cosine_similarity(np.vstack([user_vector, preprocessed_data]))
    most_similar_indices = np.argsort(-similarity_matrix[0])[1:top_n+1]
    return [data[i] for i in most_similar_indices]

# Test the recommender system
recommended_images = recommend_images(preferences, data)
for image in recommended_images:
    print(image.name)

# Test

In [ ]:
import time
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import gym
import numpy as np
from gym import spaces
import torch.nn as nn
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from sklearn.metrics.pairwise import cosine_similarity

def clean(data):
    """
    :param data: DataFrame
    :return:
    """
    # 1. first, get only the first hex_color
    # 2. Convert each column to the right type of data
    # 2. Remove rows with missing values

    data['hex_color'] = data['hex_color'].apply(lambda x: eval(x)[0] if len(x) > 0 else None)
    data['width'] = pd.to_numeric(data['width'], errors='coerce')
    data['height'] = pd.to_numeric(data['height'], errors='coerce')
    data['orientation'] = pd.to_numeric(data['orientation'], errors='coerce')
    data['width'].fillna(0, inplace=True)
    data['height'].fillna(0, inplace=True)
    data['orientation'].fillna(0, inplace=True)

    data = data.dropna()

    return data


def preprocess(data):
    # 1. Normalize and scale numerical properties
    # Width and Height: scale to [0, 1] by dividing each value by the maximum value
    # Hex colors: Convert hex colors to RGB values in the range 0-255 and normalize it by dividing by 255 (range [0, 1])

    data['width'] = data['width'] / data['width'].max()
    data['height'] = data['height'] / data['height'].max()

    # Convert hex color to RGB and normalize
    # Remove # from hex color
    data["hex_color"] = data["hex_color"].apply(lambda x: x[1:])
    data["rgb_color"] = data["hex_color"].apply(lambda x: tuple(int(x[i:i + 2], 16) for i in (0, 2, 4)))
    data[["r", "g", "b"]] = pd.DataFrame(data["rgb_color"].tolist(), index=data.index) / 255
    data.drop(["hex_color", "rgb_color"], axis=1, inplace=True)

    # One-hot encode categorical properties
    # 1. tags, make, orientation
    # eval tags to have an array
    convert_to_array = lambda tag_string: ast.literal_eval(tag_string)

    data["tags"] = data["tags"].apply(convert_to_array)
    data = pd.concat([data, pd.get_dummies(data["tags"].apply(pd.Series).stack()).sum(level=0)], axis=1)
    data = pd.concat([data, pd.get_dummies(data["make"], prefix="make")], axis=1)
    data["landscape"] = data["orientation"].apply(lambda x: 1 if x == 0 else 0)
    data["portrait"] = data["orientation"].apply(lambda x: 1 if x == 1 else 0)
    data.drop(["tags", "make", "orientation"], axis=1, inplace=True)

    return data


def train_and_test():
    df = pd.read_csv('./raw_image_properties.csv', sep="|")

    # Convert the list of ImageProperties objects to a pandas DataFrame
    #df = pd.DataFrame(raw_image_properties)

    clean_image_properties = clean(df)
    processed_image_properties = preprocess(clean_image_properties)

    # Check for NaN values in the DataFrame
    if processed_image_properties.isnull().values.any():
        processed_image_properties.fillna(0, inplace=True)

    # Create feature vectors (numpy array)
    feature_vectors = processed_image_properties.drop(["name"], axis=1).values

    # Combine the feature vectors with the image names in a new DataFrame
    data_with_features = processed_image_properties[['name']].copy()
    data_with_features['features'] = list(feature_vectors)

    # Split the dataset into training (70%) and the remaining 30%
    train_data, temp_data = train_test_split(data_with_features, test_size=0.3, random_state=42)

    # Split the remaining data into validation (15%) and test (15%) sets
    validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    env = DummyVecEnv([lambda: ImageRecommenderEnvironment(data_with_features)])

    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    policy_kwargs = dict(
        features_extractor_class=CustomDQNNetwork,
        features_extractor_kwargs={'input_dim': input_dim, 'output_dim': output_dim}
    )

    agent = DQN(MlpPolicy, env, policy_kwargs=policy_kwargs, verbose=0)

    total_timesteps = 500000  # Adjust this value based on the problem and computational resources
    print("Starting training at time", time.strftime("%H:%M:%S", time.localtime()))
    start_time = time.time()
    agent.learn(total_timesteps=total_timesteps)
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds")

    # Save the trained agent
    agent.save("dqn_agent.pkl")

    # Load the trained agent
    #agent = DQN.load("dqn_agent.pkl")

    # Create validation and test environments
    validation_env = DummyVecEnv([lambda: ImageRecommenderEnvironment(validation_data)])
    test_env = DummyVecEnv([lambda: ImageRecommenderEnvironment(test_data)])

    # Evaluate the agent on the validation and test sets
    num_validation_episodes = 100000
    num_test_episodes = 100000

    print("Starting evaluation at time", time.strftime("%H:%M:%S", time.localtime()))
    start_time = time.time()
    validation_reward, validation_duration, validation_steps = evaluate_agent(agent, validation_env, num_validation_episodes)
    test_reward, test_duration, test_steps = evaluate_agent(agent, test_env, num_test_episodes)
    end_time = time.time()
    print(f"Evaluation completed in {end_time - start_time:.2f} seconds")

    print(f"Validation reward: {validation_reward}, duration: {validation_duration:.2f}s, steps: {validation_steps}")
    print(f"Test reward: {test_reward}, duration: {test_duration:.2f}s, steps: {test_steps}")

    # Precision = TP / (TP + FP)
    # Recall = TP / (TP + FN)
    # F1-score = 2 * (Precision * Recall) / (Precision + Recall)


def preprocess_preferences(preferences, max_width, max_height, all_indexes):
        # 1. Normalize and scale numerical properties
        # Width and Height: scale to [0, 1] by dividing each value by the maximum value
        # Hex colors: Convert hex colors to RGB values in the range 0-255 and normalize it by dividing by 255 (range [0, 1])

        preferences['width'] = preferences['width'] / max_width
        preferences['height'] = preferences['height'] / max_height

        # Convert hex color to RGB and normalize
        # Remove # from hex color
        try:
            preferences["hex_color"] = preferences["hex_color"].apply(lambda x: x[1:])
            preferences["rgb_color"] = preferences["hex_color"].apply(lambda x: tuple(int(x[i:i + 2], 16) for i in (0, 2, 4)))
            preferences[["r", "g", "b"]] = pd.DataFrame(preferences["rgb_color"].tolist(), index=preferences.index) / 255
            preferences.drop(["hex_color", "rgb_color"], axis=1, inplace=True)
        except:
            pass

        # One-hot encode categorical properties
        # 1. tags, make, orientation
        # eval tags to have an array
        convert_to_array = lambda tag_string: ast.literal_eval(tag_string)

        preferences["tags"] = preferences["tags"].apply(convert_to_array)
        data = pd.concat([preferences, pd.get_dummies(preferences["tags"].apply(pd.Series).stack()).sum(level=0)], axis=1)
        data = pd.concat([data, pd.get_dummies(data["make"], prefix="make")], axis=1)
        data["landscape"] = data["orientation"].apply(lambda x: 1 if x == 0 else 0)
        data["portrait"] = data["orientation"].apply(lambda x: 1 if x == 1 else 0)
        data.drop(["tags", "make", "orientation"], axis=1, inplace=True)

        present_indexes = data.columns.values.tolist()

        # Add all indexes to the data as columns and set it to 0 if it is not present
        for index in all_indexes:
            if index not in present_indexes:
                data[index] = 0

        try:
            data.drop("hex_color")
        except:
            pass

        # if prepo_preferences do not have the same lenght as processed_image_properties remove columns that is not in processed_image_properties and add the one that are not in  prepro_preferences bt processed_image_properties

        if len(data.columns) != len(all_indexes):
            for col in data.columns:
                if col not in all_indexes:
                    data.drop(col, axis=1, inplace=True)
            for col in data.columns:
                if col not in data.columns:
                    data[col] = 0

        return data


def clean_prefs(preferences):

    try:
        preferences['hex_color'] = preferences['hex_color'].apply(lambda x: eval(x)[0] if len(x) > 0 else None)
    except:
        preferences['hex_color'] = 0

    try:
        preferences['width'] = pd.to_numeric(preferences['width'], errors='coerce')
    except:
        preferences['width'] = None

    try:
        preferences['height'] = pd.to_numeric(preferences['height'], errors='coerce')
    except:
        preferences['height'] = None

    try:
        preferences['orientation'] = pd.to_numeric(preferences['orientation'], errors='coerce')
    except:
        preferences['orientation'] = None

    preferences['width'].fillna(0, inplace=True)
    preferences['height'].fillna(0, inplace=True)
    preferences['orientation'].fillna(0, inplace=True)

    # check if there is make or others to set it to None
    return preferences


def get_image_name(image_filenames, index):
    return image_filenames[index]

def recommend():
    # TODO: convert the user preferences to clean and processed data
    # and : action, _ = agent.predict(user_preferences, deterministic=True)
    # return int(action)

    df = pd.read_csv('./raw_image_properties.csv', sep="|")

    # Convert the list of ImageProperties objects to a pandas DataFrame
    #df = pd.DataFrame(raw_image_properties)

    clean_image_properties = clean(df)
    max_width = clean_image_properties["width"].max()
    max_height = clean_image_properties["height"].max()
    processed_image_properties = preprocess(clean_image_properties)

    images_filenames = processed_image_properties["name"].values.tolist()

    tags = "['cat']"

    preferences = {
        'make': 'Apple',
        'tags': tags
    }

    df = pd.DataFrame([preferences])

    clean_preferences = clean_prefs(df)

    prepro_preferences = preprocess_preferences(clean_preferences, max_width, max_height, processed_image_properties.columns)

    # do the prediction by loading the model
    model = DQN.load("dqn_agent.pkl")
    # drop the first column
    prepro_preferences.drop(prepro_preferences.columns[0], axis=1, inplace=True)
    action, _ = model.predict(prepro_preferences, deterministic=True)

    return get_image_name(images_filenames, int(action))



class ImageRecommenderEnvironment(gym.Env):
    def __init__(self, data_with_features, max_steps=100):
        super(ImageRecommenderEnvironment, self).__init__()

        self.data = data_with_features
        self.current_state = None
        self.max_steps = max_steps
        self.step_count = 0

        # Define action space as the indices of the images
        self.action_space = spaces.Discrete(len(self.data))

        # Define state space as the feature vectors
        feature_vector_length = len(self.data['features'].iloc[0])
        self.observation_space = spaces.Box(low=0, high=1, shape=(feature_vector_length,), dtype=np.float32)

    def reset(self):
        # Reset the environment to an initial state
        self.current_state = self._get_initial_state()
        return self.current_state

    def step(self, action):
        # Execute the given action and observe the next state and reward
        next_state, reward = self._get_next_state_and_reward(action)

        # Check if the episode is done
        self.step_count += 1
        done = self._is_done(next_state)

        # Update the current state
        self.current_state = next_state

        return next_state, reward, done, {}

    def render(self, mode='human'):
        # Render the current state of the environment (optional)
        pass

    def _get_initial_state(self):
        # Randomly select an initial state from the data
        initial_state = self.data.sample().iloc[0]['features']
        return initial_state

    def _get_next_state_and_reward(self, action):
        # Limit the action value to the maximum index of the DataFrame
        action = min(action, len(self.data) - 1)

        # Get the recommended image's features based on the action
        recommended_image_features = self.data.iloc[action]['features']

        # Calculate the reward using the reward_function (you need to define user_preferences)
        user_preferences = self.current_state.reshape(1, -1)
        recommended_image_features = recommended_image_features.reshape(1, -1)
        reward = reward_function(user_preferences, recommended_image_features)

        # Get the next state (in this case, we will use the same state as the current state)
        next_state = self.current_state

        return next_state, reward


    def _is_done(self, next_state):
        # In this example, we will define a specific condition for the episode to be done.
        # If the step_count reaches the max_steps, the episode is done.
        return self.step_count >= self.max_steps

class CustomDQNNetwork(nn.Module):
    def __init__(self, *args, input_dim=None, output_dim=None, **kwargs):
        super(CustomDQNNetwork, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, output_dim)
        )

        self.features_dim = output_dim

    def forward(self, x):
        return self.network(x)

def reward_function(user_preferences, recommended_image_features):
    similarities = cosine_similarity(user_preferences, recommended_image_features)
    return similarities.mean()

import time

def evaluate_agent(agent, env, num_episodes):
    total_rewards = []
    episode_durations = []
    episode_steps = []

    for episode in range(num_episodes):
        start_time = time.time()
        state = env.reset()
        done = False
        episode_reward = 0
        step_count = 0

        while not done:
            action, _ = agent.predict(state, deterministic=True)
            next_state, reward, done, _ = env.step(action)
            episode_reward += reward
            state = next_state
            step_count += 1
            print(f"Episode {episode + 1}, Step {step_count}: Reward={reward}")


        end_time = time.time()
        episode_duration = end_time - start_time
        total_rewards.append(episode_reward)
        episode_durations.append(episode_duration)
        episode_steps.append(step_count)

        # Print progress
        print(f"Episode {episode + 1}/{num_episodes}: Reward={episode_reward}, Duration={episode_duration:.2f}s, Steps={step_count}")

    avg_reward = np.mean(total_rewards)
    avg_duration = np.mean(episode_durations)
    avg_steps = np.mean(episode_steps)

    return avg_reward, avg_duration, avg_steps


In [ ]:
train_and_test()

In [ ]:
recommend()